## Injection benchmark

CPython 3.6.3 and an i7 7700K were used for the timings

In [1]:
from antidote import antidote
import attr

container = antidote.container

In [2]:
@antidote.register
class Service1:
    pass


@antidote.register
class Service2:
    def __init__(self, service1: Service1):
        self.service1 = service1
       
 
@antidote.register
class Service3:
    def __init__(self, service1: Service1, service2: Service2):
        self.service1 = service1
        self.service2 = service2

  
@antidote.register
class Service4:
    def __init__(self, service1: Service1, service2: Service2, service3: Service3):
        self.service1 = service1
        self.service2 = service2
        self.service3 = service3


### Function call

Injection overhead is here measured with a function which does nothing.

In [3]:
def f(s1: Service1, s2: Service2, s3: Service3, s4: Service4):
    pass

Time necessary to only execute the function, without retrieving the services

In [4]:
args = (container[Service1], container[Service2], container[Service3], container[Service4])
%timeit f(*args)

63.9 ns ± 0.282 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Overhead of the injection when all argument must be retrieved from the container.

In [5]:
f_injected = antidote.inject(f)
%timeit f_injected()

1.81 µs ± 6.86 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Overhead of the injection when no argument has to be retrieved.

In [6]:
%timeit f_injected(*args)

772 ns ± 9.83 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


If speed is critical, arguments can be bound with a `functools.partial`.

In [7]:
f_bound = antidote.inject(f, bind=True)

%timeit f_bound()

186 ns ± 0.931 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


It should be noted that this is the worst scenario possible. In a real case example, the function would be much slower.
To put those results into perspective, the overhead is roughly the time needed to decode this JSON.

In [8]:
import json
%timeit json.loads('{ "name":"John", "age":30, "city":"New York"}')

1.79 µs ± 24.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Object instantiation

A similar benchmark is done with object instantiation.

In [9]:
class Obj:
    s1: Service1
    s2: Service2
    s3: Service3
    s4: Service4
        
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit Obj(*args)

255 ns ± 2.41 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
class ObjInjected:
    s1: Service1
    s2: Service2
    s3: Service3
    s4: Service4
        
    @antidote.inject
    def __init__(self, s1: Service1, s2: Service2, s3: Service3, s4: Service4):
        self.s1 = s1
        self.s2 = s2
        self.s3 = s3
        self.s4 = s4

%timeit ObjInjected()

2.09 µs ± 16.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
@attr.s
class ObjAttrs:
    s1: Service1 = antidote.attrib()
    s2: Service2 = antidote.attrib()
    s3: Service3 = antidote.attrib()
    s4: Service4 = antidote.attrib()
        
%timeit ObjAttrs()

1.51 µs ± 11.8 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


### Conclusion

The key take away from those benchmarks, is to avoid using injection for short functions which are called repeatedly, in a loop typically. In the other cases, the overhead is considered to be negligeable.

Yet should it not be the case, when measured with a profiling tool, consider to either use the option `bind=True` or to retrieve the necessary services beforehand.